In [5]:
import paho.mqtt.client as mqtt
import redis
import json
import uuid

In [ ]:
# cambia con la tua matricola
topic = 's303903'

In [6]:
REDIS_HOST = 'redis-10169.c311.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 10169
REDIS_USER = 'default'
REDIS_PASSWORD = '1nfOpBWy903d0oyagpUX7NrXNRYnwcTE'

In [8]:
class RedisClient():
    def __init__(self,host,port,username,password):
        self.host = host
        self.port = port
        self.username = username
        self.password = password
        self.mac_address = hex(uuid.getnode())
        self.redis_client = redis.Redis(host = self.host, port = self.port, username = self.username, password = self.password)


    def is_connected(self):
        is_connected = self.redis_client.ping()
        print(f'Redis Connected: {is_connected} \n')


    def create_key(self, key_name, retention_time_ms = None):
        try:
            self.redis_client.ts().create(key_name, retention_msecs=retention_time_ms)
        except redis.ResponseError:
            pass

    def add_ts_battery_data(self, message : str):
        data = json.loads(message)
        mac_address = data['mac_address']

        for i in range(len(data['events'])):
            dic = data['events'][i]
            timestamp_ms = int(dic['timestamp'] * 1000)
            battery = dic['battery_level']
            power_plugged = dic['power_plugged']
            self.redis_client.ts().add(f'{mac_address}:battery', timestamp_ms, battery)
            self.redis_client.ts().add(f'{mac_address}:power', timestamp_ms, power_plugged)


    

In [9]:
client = mqtt.Client()
redis_client = RedisClient(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USER, password=REDIS_PASSWORD)
redis_client.is_connected()
mac_address = redis_client.mac_address
redis_client.create_key(f'{mac_address}:battery', retention_time_ms= 24 * 60 * 60 * 1000)
redis_client.create_key(f'{mac_address}:power', retention_time_ms=24 * 60 * 60 * 1000)

Redis Connected: True 



In [10]:
def on_connect(client: mqtt.Client, userdata, flags, rc):
    print(f'Connected with result code {str(rc)}')
    client.subscribe('s303903')

def on_message(client, userdata, msg):
    message = msg.payload.decode()
    topic = msg.topic
    redis_client.add_ts_battery_data(message)
    print(f'Received {message} on topic {topic}')

client.on_connect = on_connect
client.on_message = on_message

client.connect('mqtt.eclipseprojects.io', 1883)

client.loop_forever()

Connected with result code 0
Received {"mac_address": "0xacde48001122", "events": [{"timestamp": 1705932837645, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932838648, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932839651, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932840656, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932841660, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932842666, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932843671, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932844676, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932845680, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932846684, "battery_level": 100, "power_plugged": 1}]} on topic s303903
Received {"mac_address": "0xacde48001122", "events": [{"timestamp": 1705932847687, "battery_level": 100, "power_plugged": 1}, {"timestamp": 1705932848692, "battery_level": 100, "p

KeyboardInterrupt: 